<a href="https://colab.research.google.com/github/Kaiziferr/Deep_Learning_Workshop/blob/master/multilayer_perceptron/07_workshop_learningrate_adaptative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import tensorflow

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

from tensorflow.keras.metrics import Recall


from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import math

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv')
data_values = data.values

# Split data

In [3]:
X = data_values[:,0:34].astype('float')
y = data_values[:,34]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.85, shuffle = True)

In [5]:
# Encoder
encoder = LabelEncoder()
y_train_encoder = encoder.fit_transform(y_train)
y_test_encoder = encoder.transform(y_test)

In [6]:
#escalar
estimator_scaler = StandardScaler()
X_train_scaler = estimator_scaler.fit_transform(X_train)
X_test_scaler = estimator_scaler.transform(X_test)

In [7]:
data['g'].value_counts()

g    224
b    126
Name: g, dtype: int64

# Optimizador Adaptativo Adam

In [12]:
model = Sequential()
model.add(Dense(34, input_dim =34, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=[Recall()])
model.fit(X_train_scaler, y_train_encoder, validation_data=(X_test_scaler, y_test_encoder), epochs = 50, batch_size = 28, verbose=1)

Epoch 1/50
11/11 [==============================] - 1s 23ms/step - loss: 0.6742 - recall_1: 0.4149 - val_loss: 0.6029 - val_recall_1: 0.6389
Epoch 2/50
11/11 [==============================] - 0s 4ms/step - loss: 0.5688 - recall_1: 0.7713 - val_loss: 0.5077 - val_recall_1: 0.8333
Epoch 3/50
11/11 [==============================] - 0s 4ms/step - loss: 0.4933 - recall_1: 0.8936 - val_loss: 0.4396 - val_recall_1: 0.8333
Epoch 4/50
11/11 [==============================] - 0s 3ms/step - loss: 0.4395 - recall_1: 0.9096 - val_loss: 0.3916 - val_recall_1: 0.8333
Epoch 5/50
11/11 [==============================] - 0s 3ms/step - loss: 0.3985 - recall_1: 0.9149 - val_loss: 0.3550 - val_recall_1: 0.8611
Epoch 6/50
11/11 [==============================] - 0s 3ms/step - loss: 0.3689 - recall_1: 0.9202 - val_loss: 0.3264 - val_recall_1: 0.8611
Epoch 7/50
11/11 [==============================] - 0s 3ms/step - loss: 0.3435 - recall_1: 0.9255 - val_loss: 0.3052 - val_recall_1: 0.8611
Epoch 8/50
11/11 [=

# Planificación de learning rate en función del tiempo

In [8]:
model = Sequential()
model.add(Dense(34, input_dim = 34, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid'))


epoch = 50
lr = 0.1
decay_rate = lr/epoch
momentum = 0.8

sgd = SGD(learning_rate=lr, momentum=momentum, decay = decay_rate, nesterov = False)
model.compile(loss='binary_crossentropy', optimizer = sgd, metrics = [Recall()])

In [9]:
model.fit(X_train_scaler,y_train_encoder, validation_data=(X_test_scaler, y_test_encoder), epochs=epoch, batch_size=28, verbose = 1)

Epoch 1/50
11/11 [==============================] - 1s 26ms/step - loss: 0.4883 - recall: 0.7606 - val_loss: 0.2364 - val_recall: 0.8889
Epoch 2/50
11/11 [==============================] - 0s 3ms/step - loss: 0.2493 - recall: 0.9415 - val_loss: 0.1561 - val_recall: 0.9722
Epoch 3/50
11/11 [==============================] - 0s 4ms/step - loss: 0.1824 - recall: 0.9628 - val_loss: 0.1211 - val_recall: 1.0000
Epoch 4/50
11/11 [==============================] - 0s 3ms/step - loss: 0.1469 - recall: 0.9681 - val_loss: 0.1005 - val_recall: 1.0000
Epoch 5/50
11/11 [==============================] - 0s 4ms/step - loss: 0.1257 - recall: 0.9840 - val_loss: 0.0975 - val_recall: 1.0000
Epoch 6/50
11/11 [==============================] - 0s 3ms/step - loss: 0.1098 - recall: 0.9787 - val_loss: 0.0790 - val_recall: 1.0000
Epoch 7/50
11/11 [==============================] - 0s 3ms/step - loss: 0.0945 - recall: 0.9894 - val_loss: 0.0744 - val_recall: 1.0000
Epoch 8/50
11/11 [=============================

#Tasa de aprendizaje basado en caidas (drop)

In [28]:
def step_decay(epoch):
  initial_rate = 0.1
  drop = 0.5
  epochs_drop = 10
  lrate = initial_rate * math.pow(drop, math.floor(1+epoch) / epochs_drop)
  print(lrate, epoch)
  return lrate

In [29]:
model = Sequential()
model.add(Dense(34, input_dim = 34, activation = 'relu', kernel_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid'))

In [30]:
sgd = SGD(learning_rate=0.0, momentum = 0.9)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=[Recall()])

In [31]:
lrate = LearningRateScheduler(step_decay)
callback_list = [lrate]

In [32]:
model.fit(X_train_scaler, y_train_encoder, validation_data=(X_test_scaler, y_test_encoder), epochs=50, batch_size=25, callbacks=callback_list
          , verbose = 1)

Epoch 1/50
0.09330329915368074 0
12/12 [==============================] - 1s 19ms/step - loss: 0.5129 - recall_4: 0.8404 - val_loss: 0.2009 - val_recall_4: 1.0000
Epoch 2/50
0.08705505632961241 1
12/12 [==============================] - 0s 3ms/step - loss: 0.2684 - recall_4: 0.9149 - val_loss: 0.1001 - val_recall_4: 1.0000
Epoch 3/50
0.08122523963562356 2
12/12 [==============================] - 0s 3ms/step - loss: 0.1611 - recall_4: 0.9574 - val_loss: 0.1246 - val_recall_4: 0.9722
Epoch 4/50
0.0757858283255199 3
12/12 [==============================] - 0s 3ms/step - loss: 0.1219 - recall_4: 0.9681 - val_loss: 0.0684 - val_recall_4: 1.0000
Epoch 5/50
0.07071067811865477 4
12/12 [==============================] - 0s 3ms/step - loss: 0.1023 - recall_4: 0.9947 - val_loss: 0.0635 - val_recall_4: 1.0000
Epoch 6/50
0.06597539553864472 5
12/12 [==============================] - 0s 3ms/step - loss: 0.0803 - recall_4: 0.9947 - val_loss: 0.0564 - val_recall_4: 1.0000
Epoch 7/50
0.061557220667245

- Incrementar la tasa de aprendizaje inicial, debido a que la tasa de aprendizaje disminuira, inicie con un valor mayor desde el cual disminuir

- Utilizar un gran momentum

- Experimente con diferentes planificacion